In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install detection_datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [ ]:
pip install tifffile==2024.4.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tifffile
    Found existing installation: tifffile 2024.2.12
    Uninstalling tifffile-2024.2.12:
      Successfully uninstalled tifffile-2024.2.12


In [ ]:
from datasets import load_dataset, concatenate_datasets
from detection_datasets import DetectionDataset
import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import io
from PIL import Image, ImageDraw
import os
from tqdm import tqdm
from PIL import Image
from datasets import Dataset
import io
import os

### Process the below code block to get train parquet files, and test parquet
#### Adjust the iteration accordingly to the number of parquet files needed.
##### For me, 4 parquet files for train and 1 parquet fiel for test worked seemlessly

In [ ]:
import requests

# Function to fetch dataset from URL
def fetch_dataset(dataset_url):
    response = requests.get(dataset_url)
    if response.status_code == 200:
        return response.text
    else:
        print("Failed to fetch the dataset.")
        return None

# Example URL of the dataset
dataset_url = "https://huggingface.co/datasets/back2classroom/sidewalks"

# Fetching the dataset content
dataset_content = fetch_dataset(dataset_url)

# Displaying the fetched content
print(dataset_content)


# Continue with processing the dataset if it was fetched successfully
if df is not None:
    dataset_path = "./Downloads/"

    train_path = dataset_path + "Train/"
    test_path = dataset_path + "Test/"

    train_parquet_path = dataset_path + "Parquets/Train/"

    if not os.path.exists(train_path):
        print("Train directory does not exist.")
        exit()

    train_files = os.listdir(train_path)

    for i in range(4):
        new_df = pd.DataFrame(columns=["filename", "tif", "tfw"])
        files = train_files[90000*i:min(90000*(i+1), len(train_files))]  # Ensure not to go out of index
        for instance in tqdm(files):
            if instance.endswith(".tif"):
                im = Image.open(os.path.join(train_path, instance))

                image_bytes = io.BytesIO()
                im.save(image_bytes, format="JPEG")
                image_bytes.seek(0)

                tfw_file_path = os.path.join(train_path, instance[:-4] + ".tfw")
                if os.path.exists(tfw_file_path):
                    with open(tfw_file_path, 'rb') as tfw_file:
                        tfw_bytes = tfw_file.read()

                    new_data = {"filename": instance[:-4], "tif": image_bytes.getvalue(), "tfw": tfw_bytes}
                    new_data_df = pd.DataFrame([new_data])
                    new_df = pd.concat([new_df, new_data_df], ignore_index=True)
                else:
                    print(f"No tfw file found for {instance}")

        new_df.to_parquet(train_parquet_path + f'train{i}.parquet')


#### Code to check the the parquet file is being processed correctly

In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import requests
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Function to fetch Parquet file from URL
def fetch_parquet_from_url(parquet_url):
    response = requests.get(parquet_url)
    if response.status_code == 200:
        # Write the Parquet content to a BytesIO object
        parquet_bytes = io.BytesIO(response.content)
        return parquet_bytes
    else:
        print("Failed to fetch the Parquet file.")
        return None

# Example URL of the Parquet file
parquet_url = "https://huggingface.co/datasets/back2classroom/sidewalks.parquet"

# Fetching the Parquet file content
parquet_bytes = fetch_parquet_from_url(parquet_url)

# If the Parquet file content was fetched successfully
if parquet_bytes is not None:
    # Save the Parquet content to a temporary file
    with open("temp.parquet", "wb") as f:
        f.write(parquet_bytes.getvalue())

    # Read the Parquet file into a pandas DataFrame
    df = pd.read_parquet("temp.parquet")

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        # Extract image data from the DataFrame
        image_data = row['tif']
        im = Image.open(io.BytesIO(image_data))

        # Display the image
        plt.imshow(np.asarray(im))
        plt.show()

        # Extract and display tfw information
        tfw_bytes = row['tfw']
        tfw = tfw_bytes.decode('utf-8')
        print("TFW information:")
        print(tfw)

        # Split tfw information into individual elements
        tfw_elements = tfw.split()
        print("TFW elements:")
        print(tfw_elements)

        # Break after processing the first row (remove this line if you want to process all rows)
        break

Failed to fetch the Parquet file.


# Place the hugging face token in the below command to write to datasets

In [12]:
!huggingface-cli login --token [token_value]

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/user.py", line 98, in run
    login(token=self.args.token, add_to_git_credential=self.args.add_to_git_credential)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py", line 110, in login
    _login(token, add_to_git_credential=add_to_git_credential, write_permission=write_permission)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py", line 306, in _login
    raise ValueError("Invalid token passed!")
ValueError: Invalid token passed!


In [4]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
train_parquet_path = "https://huggingface.co/datasets/back2classroom/sidewalks/"


In [5]:
import tifffile
import pandas as pd
from datasets import Dataset

# Define the path to the Parquet files
train_parquet_path = "https://huggingface.co/datasets/back2classroom/sidewalks/"


# Iterate over the Parquet files
for i in range(4):
    # Construct the file path for the current Parquet file
    parquet_file_path = train_parquet_path + 'train' + str(i) + '.parquet'

    try:
        # Read the Parquet file into a pandas DataFrame
        df = pd.read_parquet(parquet_file_path)

        # Convert the DataFrame into a Hugging Face Dataset
        dataset = Dataset.from_pandas(df)

        # Push the dataset to the Hugging Face Dataset Hub
        dataset.push_to_hub("back2classroom/sidewalks_chunk" + str(i+1), split="train")

        print(f"Dataset for chunk {i+1} pushed successfully.")

    except Exception as e:
        print(f"Failed to push dataset for chunk {i+1} due to error: {e}")

Failed to push dataset for chunk 1 due to error: HTTP Error 401: Unauthorized
Failed to push dataset for chunk 2 due to error: HTTP Error 401: Unauthorized
Failed to push dataset for chunk 3 due to error: HTTP Error 401: Unauthorized
Failed to push dataset for chunk 4 due to error: HTTP Error 401: Unauthorized


# Pushing Test dataset to val split

In [15]:
## change the path to location of test parquet file
from datasets import load_dataset
dataset = load_dataset("back2classroom/sidewalks", split='val', streaming=True)
print(next(iter(dataset)))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'filename': '1016_10_1', 'tfw': b'1.0000000000\r\n0.0000000000\r\n0.0000000000\r\n1.0000000000\r\n0.0000000000\r\n2304.0000000000\r\n', 'tif': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01\x00\x01\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\

# Pushing Train dataset to train split

In [16]:
from datasets import load_dataset

dataset_chunks = []

dataset_chunk = load_dataset("back2classroom/sidewalks_chunk1")
dataset_chunks.append(dataset_chunk['train'])

dataset_chunk = load_dataset("back2classroom/sidewalks_chunk2")
dataset_chunks.append(dataset_chunk['train'])

dataset_chunk = load_dataset("back2classroom/sidewalks_chunk3")
dataset_chunks.append(dataset_chunk['train'])

dataset_chunk = load_dataset("back2classroom/sidewalks_chunk4")
dataset_chunks.append(dataset_chunk['train'])

# Concatenate datasets
concatenated_dataset = concatenate_datasets(dataset_chunks)



Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

In [17]:
print(concatenated_dataset)

Dataset({
    features: ['filename', 'tfw', 'tif', 'label_tif', 'label_tfw'],
    num_rows: 180000
})
